In [1]:
from datasets import load_dataset, load_from_disk
from promptsource import templates

CACHE_DIR = "/share/edc/home/antonis/datasets/huggingface"
import os
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR

# # Get a list of all supported datasets
# datasets = templates.get_dataset_names()
# print(datasets)

/local/home/antonis/.conda/envs/incidental/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def print_example(example):
    for key in example.keys():
        print(f"{key}: {example[key]}")

In [3]:
# import sys
# sys.path.append("/share/edc/home/antonis/LLM-Incidental-Supervision/incidental-supervision/src")
from src.dataset_configs import DatasetConfig

config_instance = DatasetConfig()

# Changing P_QA and updating 'p' values in dataset_configs
config_instance.update_p_values(new_value=0.5)

# Now, the 'p' values inside config_instance.dataset_configs are updated
print(config_instance.dataset_configs)

{'struct2text': [{'dataset_type': 'QA', 'dataset_name': 'common_gen', 'dataset_config_name': 'common_gen', 'p': 0.5, 'train_split': 'train', 'columns': {'concepts': 'concepts', 'target': 'target'}}, {'dataset_type': 'QA', 'dataset_name': 'e2e_nlg', 'dataset_config_name': 'e2e_nlg', 'p': 0.5, 'train_split': 'train', 'columns': {'meaning_representation': 'meaning_representation', 'human_reference': 'human_reference'}}, {'dataset_type': 'QA', 'dataset_name': 'dart', 'dataset_config_name': 'dart', 'p': 0.5, 'validation_split': 'validation', 'columns': {'tripleset': 'text', 'annotations': 'target'}}, {'dataset_type': 'QA', 'dataset_name': 'web_nlg', 'dataset_config_name': 'release_v3.0_en', 'p': 0.5, 'validation_split': 'test', 'columns': {'modified_triple_sets': 'modified_triple_sets', 'text': 'text'}}, {'dataset_type': 'text', 'dataset_name': 'wikitext', 'dataset_config_name': 'wikitext-2-v1', 'p': 1, 'train_split': 'train'}, {'dataset_type': 'text', 'dataset_name': 'bookcorpus', 'dataset

In [14]:
dataset = load_dataset('web_nlg', 'release_v3.0_en', cache_dir=CACHE_DIR)

Found cached dataset web_nlg (/share/edc/home/antonis/datasets/huggingface/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)
100%|██████████| 3/3 [00:00<00:00, 238.88it/s]


In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 13211
    })
    dev: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 1667
    })
    test: Dataset({
        features: ['category', 'size', 'eid', 'original_triple_sets', 'modified_triple_sets', 'shape', 'shape_type', 'lex', 'test_category', 'dbpedia_links', 'links'],
        num_rows: 5713
    })
})

: 

In [4]:
ds_pth = "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_train.arrow"
dataset = load_from_disk(ds_pth)

In [5]:
dataset

Dataset({
    features: ['text', 'date', 'user', 'sentiment', 'query', 'idx', 'sentence', 'label'],
    num_rows: 2319
})

In [6]:
ds_dart = load_dataset('web_nlg', 'release_v3.0_en')

Found cached dataset web_nlg (/local/home/antonis/.cache/huggingface/datasets/web_nlg/release_v3.0_en/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda)
100%|██████████| 3/3 [00:00<00:00, 124.34it/s]


In [7]:
dataset_files = [
    "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_1/dataset_train.arrow",
    "/share/edc/home/antonis/datasets/huggingface/merged_datasets/dataset_0/dataset_train.arrow",
]

for ds_file in dataset_files:
    ds = load_from_disk(ds_file)
    print(len(ds))

74040946
74040946


In [8]:
imdb_dataset = load_dataset('imdb', None, split='test')
print(len(imdb_dataset))
print_example(imdb_dataset[1001])

Found cached dataset imdb (/local/home/antonis/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


25000
text: This experimental silent film, made in Switzerland by an independent British film company, is chiefly remembered as Paul Robeson's first film. It's very artistic, with shots often seeming meaningless to the story, which is difficult to understand anyway because of the lack of enough inter-titles. From what I gathered, Robeson's wife, Adah, is in an inter-racial love affair with a white man called Thorne. It doesn't bother the cigar-chomping owner of the bar/hotel where Thorne lives (and she seems to be having a lesbian relationship with a barmaid), but an old lady expresses the town's point of view in an inter-title: "If I had my way, we wouldn't allow negroes in here." Thorne is also called "nigger lover" by someone in the bar. Adah tries a reconciliation with Pete (Robeson), but eventually leaves him. Thorne's wife, Astrid, goes off the deep end, brandishes a knife, cuts Thorne's arm and cheek, and somehow dies. Thorne must have been accused of murder because we learn he 

In [9]:
sentiment_140 = load_dataset('sentiment140', None, split='test')
print(len(sentiment_140))
print_example(sentiment_140[1])

Found cached dataset sentiment140 (/local/home/antonis/.cache/huggingface/datasets/sentiment140/sentiment140/1.0.0/f81c014152931b776735658d8ae493b181927de002e706c4d5244ecb26376997)


498
text: Reading my kindle2...  Love it... Lee childs is good read.
date: Mon May 11 03:18:03 UTC 2009
user: vcu451
sentiment: 4
query: kindle2


In [10]:
sst2 = load_dataset('sst2', None, split='train')
print(len(sst2))
print_example(sst2[1])

Found cached dataset sst2 (/local/home/antonis/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


67349
idx: 1
sentence: contains no wit , only labored gags 
label: 0


In [11]:
sst2

Dataset({
    features: ['idx', 'sentence', 'label'],
    num_rows: 67349
})

In [13]:
yelp_review_full = load_dataset('yelp_review_full', None, split='test')
print(len(yelp_review_full))
print_example(yelp_review_full[-20000])

Found cached dataset yelp_review_full (/local/home/antonis/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


50000
label: 3
text: I would say I've probably come here a good 50 times mainly due to this being one of the 3 places my old boss wanted to come to for late night eating. I used to think it was alright, not too much flavor but recently I noticed a change. Although the portions got slightly smaller and prices went up $1-3 per item, the flavor was definitely kicked up! They have live music most nights and I like the atmosphere.\nMy favorites have been the Tom Kha, Panang Curry, Pad See Ew, Basil Fried Rice, Beef Salad, Garlic Shrimps, and their roti w/ coconut ice cream yum! Oh and also their Chang beer towers are a great deal at $18. Also, love their Thai tea bombs!!.\nThe items I didn't care for were their Cornish hen appetizer (hugeee but little flavor), chicken satay, and  thai bbq chicken. \nEverytime I've come in a large group of around 10 people (probably 35/50 times I've come here), we've been seated immediately and food has come fast. \nOverall I really enjoy this place especial

In [14]:
yelp_review_full

Dataset({
    features: ['label', 'text'],
    num_rows: 50000
})

: 